In [27]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import numpy as np
import cv2
from PIL import Image



In [28]:
#Initializations
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torchvision.models.segmentation.fcn_resnet50(pretrained=True).to(device)

#has to normalize data the same way the pretrained images were
normalize = transforms.Normalize(mean=[0.485,0.456,0.406],
                        std=[0.229,0.224,0.225])
data_transforms = {
    'train':
    transforms.Compose([
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.ToTensor(),
        normalize
    ]),
}

image_datasets = {
    'train': 
    
    'validation':
}

dataloaders = {
    'train':
    
    'validation':
}

for param in model.parameters():
    param.requires_grad = False   
    
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 2)).to(device)

loss_fn = nn.CrossEntropyLoss()
optim = optim.Adam((model.fc.parameters))

SyntaxError: invalid syntax (3330783709.py, line 24)

In [25]:
def train_model(model, criterion, optimizer, num_epochs=3):
    for epoch in range(num_epochs):
        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            for batch, (input,labels) in enumerate(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
    return model


In [26]:
model_trained = train_model(model, loss_fn, optim, num_epochs=3)
!mkdir models
!mkdir models/pytorch
torch.save(model_trained.state_dict(), 'models/pytorch/weights.h5')





NameError: name 'loss_fn' is not defined

In [ ]:
model = torchvision.models.segmentation.fcn_resnet50(pretrained=False).to(device)
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 2)).to(device)
model.load_state_dict(torch.load('models/pytorch/weights.h5'))